In [1]:
!pip install torchsummaryX --quiet

In [2]:
!nvidia-smi

Sat Nov 12 02:44:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
#from torchsummaryX import summary
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import os
import torchvision
from matplotlib import image,pyplot

In [53]:
!unzip -qo 'sonar_data3.zip'

In [208]:
transforms=torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
flip_transforms=torchvision.transforms.Compose([torchvision.transforms.RandomHorizontalFlip(p=1),torchvision.transforms.ToTensor()])
train_dataset = torchvision.datasets.ImageFolder(root='sonar_data2/train',transform=transforms)
train_flip_dataset=torchvision.datasets.ImageFolder(root='sonar_data2/train',transform=flip_transforms)
val_dataset = torchvision.datasets.ImageFolder(root='sonar_data2/val',transform=transforms)
train_dataset_new=torch.utils.data.ConcatDataset([train_dataset,train_flip_dataset])
#val_dataset_new=torch.utils.data.ConcatDataset([val_dataset,train_flip_dataset])

In [209]:

train_loader = torch.utils.data.DataLoader(train_dataset_new, batch_size = 16, 
                                           shuffle = True,num_workers = 2, pin_memory = True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = 16, 
                                         shuffle = False, num_workers = 2)

In [165]:
for i,data in train_loader:
  print(i.shape)
  break

torch.Size([16, 3, 256, 256])


In [166]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [210]:
class Network(torch.nn.Module):

    def __init__(self, num_classes=3):
        super().__init__()
        
        self.net=nn.Sequential(nn.Conv2d(3,32,3,2),
                               nn.BatchNorm2d(32),
                               nn.ReLU(),
                               nn.Conv2d(32,64,2,2),
                               nn.BatchNorm2d(64),
                               nn.ReLU(),
                               nn.Conv2d(64,64,2,2),
                               nn.BatchNorm2d(64),
                               nn.ReLU(),
                               nn.Conv2d(64,64,2,2),
                               nn.BatchNorm2d(64),
                               nn.ReLU(),
                               torch.nn.Flatten())

        self.cls_layer = torch.nn.Linear(14400,num_classes)#TODO
        
        
    
    def forward(self, x, return_feats=False):
        """
        What is return_feats? It essentially returns the second-to-last-layer
        features of a given image. It's a "feature encoding" of the input image,
        and you can use it for the verification task. You would use the outputs
        of the final classification layer for the classification task.

        You might also find that the classification outputs are sometimes better
        for verification too - try both.
        """
        
        feats=self.net(x)
        out=self.cls_layer(feats)
        return out
        
            
model = Network().to(device)
summary(model,(3,256,256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 127, 127]             896
       BatchNorm2d-2         [-1, 32, 127, 127]              64
              ReLU-3         [-1, 32, 127, 127]               0
            Conv2d-4           [-1, 64, 63, 63]           8,256
       BatchNorm2d-5           [-1, 64, 63, 63]             128
              ReLU-6           [-1, 64, 63, 63]               0
            Conv2d-7           [-1, 64, 31, 31]          16,448
       BatchNorm2d-8           [-1, 64, 31, 31]             128
              ReLU-9           [-1, 64, 31, 31]               0
           Conv2d-10           [-1, 64, 15, 15]          16,448
      BatchNorm2d-11           [-1, 64, 15, 15]             128
             ReLU-12           [-1, 64, 15, 15]               0
          Flatten-13                [-1, 14400]               0
           Linear-14                   

In [211]:
criterion = torch.nn.CrossEntropyLoss() # TODO: What loss do you need for a multi class classification problem?
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
# scaler = torch.cuda.amp.GradScaler()

In [212]:
def train(model, dataloader, optimizer, criterion):
    
    model.train()

    # Progress Bar 
    batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, leave=False, position=0, desc='Train', ncols=5) 
    
    num_correct = 0
    total_loss = 0

    for i, (images, labels) in enumerate(dataloader):
        
        optimizer.zero_grad() # Zero gradients

        images, labels = images.to(device), labels.to(device)
        
        with torch.cuda.amp.autocast(): # This implements mixed precision. Thats it! 
            outputs = model(images)
            #print(outputs.shape)
            #print(labels.shape)
            loss = criterion(outputs, labels)

        # Update no. of correct predictions & loss as we iterate
        num_correct += int((torch.argmax(outputs, axis=1) == labels).sum())
        total_loss += float(loss.item())

        # tqdm lets you add some details so you can monitor training as you train.
        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * num_correct / (16*(i + 1))),
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            num_correct=num_correct,
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        
        loss.backward()
        optimizer.step()

        # TODO? Depending on your choice of scheduler,
        # You may want to call some schdulers inside the train function. What are these?
      
        batch_bar.update() # Update tqdm bar

    batch_bar.close() # You need this to close the tqdm bar

    acc = 100 * num_correct / (16* len(dataloader))
    total_loss = float(total_loss / len(dataloader))

    return acc, total_loss

In [213]:
def validate(model, dataloader, criterion):
  
    model.eval()
    batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, position=0, leave=False, desc='Val', ncols=5)

    num_correct = 0.0
    total_loss = 0.0

    for i, (images, labels) in enumerate(dataloader):
        
        # Move images to device
        images, labels = images.to(device), labels.to(device)
        
        # Get model outputs
        with torch.inference_mode():
            outputs = model(images)
            loss = criterion(outputs, labels)

        num_correct += int((torch.argmax(outputs, axis=1) == labels).sum())
        total_loss += float(loss.item())

        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * num_correct / (16*(i + 1))),
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            num_correct=num_correct)
        batch_bar.update()
        
    batch_bar.close()
    acc = 100 * num_correct / (16* len(dataloader))
    total_loss = float(total_loss / len(dataloader))
    return acc, total_loss

In [214]:
import gc
gc.collect() # These commands help you when you face CUDA OOM error
torch.cuda.empty_cache()

In [215]:
train_accuracies=[]
train_losses=[]
val_accuracies=[]
val_losses=[]

In [216]:

for epoch in range(0,20):

    train_acc, train_loss = train(model, train_loader, optimizer, criterion)
    
    print("\nEpoch {}/{}: \nTrain Acc {:.04f}%\t Train Loss {:.04f}\t".format(
        epoch + 1,
        20,
        train_acc,
        train_loss))
    
    val_acc, val_loss = validate(model, val_loader, criterion)
    print("Val Acc {:.04f}%\t Val Loss {:.04f}".format(val_acc, val_loss))
    train_accuracies.append(train_acc)
    train_losses.append(train_loss)
    val_accuracies.append(val_acc)
    val_losses.append(val_loss)



Epoch 1/20: 
Train Acc 75.0601%	 Train Loss 0.6557	


Val Acc 89.3750%	 Val Loss 0.3857



Epoch 2/20: 
Train Acc 85.5769%	 Train Loss 0.4160	


Val Acc 90.0000%	 Val Loss 0.3191



Epoch 3/20: 
Train Acc 87.4399%	 Train Loss 0.3423	


...

Val Acc 91.8750%	 Val Loss 0.2155



Epoch 19/20: 
Train Acc 97.1755%	 Train Loss 0.1107	


Val Acc 91.8750%	 Val Loss 0.2113



Epoch 20/20: 
Train Acc 97.4760%	 Train Loss 0.1064	


Val Acc 91.8750%	 Val Loss 0.2160
